# ADM - HW 3

# 1. Data Collection

## 1.1 Get the list of master's degree courses

In [ ]:
if False: 
    pip install bs4
    pip install selenium

In [9]:
from bs4 import BeautifulSoup
import requests
import os 

First of all, we get a list of all the master's degree courses we need.

In [17]:
def extract_masters(this_url):
  result_url = requests.get(this_url)
  result_soup = BeautifulSoup(result_url.text)
  result_links = result_soup.find_all('a', {'class':'courseLink'})
  result_list=[]
  for item in result_links:
    result_list.append((item['href'], item.text))

  return result_list

We notice that the url of th various pages is *https://www.findamasters.com/masters-degrees/msc-degrees/?PG=* followed by the number of the page in the MSc courses catalogue we're in. 

In [ ]:
links = []
for page in range(1, 401):
    url = 'https://www.findamasters.com/masters-degrees/msc-degrees/?PG='+str(page)
    links += extract_masters(url)

We want to verify we retrieved all of the needed urls, so we check for the length of the list

In [22]:
print(len(links))

6000


It's the expected length, so we can create and write the required *.txt* file.   
Before doing this, I created a new empty file called "msc_links" and saved it in the folder where I'm currently working.

In [23]:
file = open('msc_links.txt','w')
for link in links:
	file.write(link[0]+"\n")
file.close()

Now, if we look at the file it's written in the correct way (:

## 1.2 Crawl master's degree pages 

In [70]:
def download_html(url, dest_folder):
    #path is the local path we're working in 
    if not os.path.exists(dest_folder):  # create folder if it does not exist
        os.makedirs(dest_folder) 
    
    url = 'https://www.findamasters.com/'+url[:-2]
    filename = url.replace('https://www.findamasters.com//masters-degrees/course/', '').replace('/', '_')
    file_path = os.path.join(dest_folder, filename)
    url_req = requests.get(url)

    soup = BeautifulSoup(url_req.text, 'html.parser')
    with open(file_path, 'w') as file: 
        file.write(soup.prettify())
    
    file.close()
    

In [ ]:
### SAVING HTML & ORGANIZING THEM IN FOLDERS ###
import itertools

prova = int(input())
with open('msc_links.txt', 'r') as file:
    for i in range(1, prova):
        folder_i = 'page_' + str(i)
        
        #selecting the lines corresponding to the i-th page
        needed_lines = itertools.islice(file, 0, 15)

        #saving the corresponding html in "page_i" folder
        for url in needed_lines:
            download_html(str(url), folder_i)